<a href="https://colab.research.google.com/github/Nadia147/ML_boundary/blob/main/assig_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install earthpy
# Colab Requirements
!pip install spectral
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
## Basics
import gc
gc.collect()
import warnings
warnings.filterwarnings('ignore')
import time
import numpy
import numpy as np
from PIL import Image
from operator import truediv
import scipy.io as sio
import os
import pandas as pd
import seaborn as sns
import spectral
## Ploting
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
%matplotlib inline
## Sklearn
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from sklearn.decomposition import IncrementalPCA
## Deep Model
import keras
import h5py
from keras.layers import Dropout, Input, Conv2D, Conv3D, MaxPool3D, Flatten, Dense, Reshape, BatchNormalization
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.models import Sequential, Model
#from keras.utils import np_utils
from keras.utils import to_categorical

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
## Mounting Colab
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import matplotlib.pyplot as plt

import numpy as np
import earthpy.plot as ep
import seaborn as sns
import earthpy.spatial as es

import plotly.graph_objects as go
import plotly.express as px

from scipy.io import loadmat

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, classification_report)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tqdm import tqdm
import scipy.io

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
from google.colab import drive
drive.mount('/content/drive')

# Define file paths for your data directories
train_dir = '/content/drive/MyDrive/Colab Notebooks/train'
label_dir = '/content/drive/MyDrive/Colab Notebooks/labels'
test_dir = '/content/drive/MyDrive/Colab Notebooks/private'

train_path = '/content/drive/MyDrive/Colab Notebooks/train'
valid_path = '/content/drive/MyDrive/Colab Notebooks/private'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


94765736/94765736 [==============================] - 4s 0us/step


In [7]:


# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False



In [8]:


  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Colab Notebooks/train/*')



In [9]:


# our layers - you can add more if you want
x = Flatten()(resnet.output)



In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
#model = Model(inputs=resnet.input, outputs=prediction)


# view the structure of the model
#model.summary()



In [11]:
import scipy.io
from sklearn.model_selection import train_test_split
#from keras.utils import np_utils
from keras.layers import Input, Conv3D,Conv2D, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
#import time
# Model Structure
input_layer = resnet.input
conv_layer1 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(input_layer)


flatten_layer = Flatten()(conv_layer1)
dense_layer1 = Dense(units=6, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=6, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=6, activation='softmax')(dense_layer2)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 8)       224       
                                                                 
 flatten_1 (Flatten)         (None, 394272)            0         
                                                                 
 dense_1 (Dense)             (None, 6)                 2365638   
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0     

In [12]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (256, 256),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 760 images belonging to 6 classes.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')

# Define the data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,     # Normalize pixel values to [0, 1]
    shear_range=0.2,    # Shear transformations
    zoom_range=0.2,     # Zoom transformations
    horizontal_flip=True)  # Horizontal flips

# Provide the path to your dataset directory in Colab
dataset_path = '/content/drive/MyDrive/Colab Notebooks/train'  # Change this to your actual dataset path

# Create a generator for the training set
training_set = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Set your desired target size
    batch_size=16,
    class_mode='categorical'
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 760 images belonging to 6 classes.


In [16]:
test_set = test_datagen.flow_from_directory(dataset_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 760 images belonging to 6 classes.


In [ ]:


# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)



Epoch 1/30
48/48 [==============================] - 344s 7s/step - batch: 23.5000 - size: 15.8333 - loss: 5.3926 - acc: 0.2658 - val_loss: 1.7660 - val_acc: 0.2684
Epoch 2/30
48/48 [==============================] - 13s 273ms/step - batch: 23.5000 - size: 15.8333 - loss: 1.7540 - acc: 0.2618 - val_loss: 1.7411 - val_acc: 0.2684
Epoch 3/30
48/48 [==============================] - 13s 266ms/step - batch: 23.5000 - size: 15.8333 - loss: 1.7299 - acc: 0.2684 - val_loss: 1.7175 - val_acc: 0.2684
Epoch 4/30
48/48 [==============================] - 13s 266ms/step - batch: 23.5000 - size: 15.8333 - loss: 1.7070 - acc: 0.2605 - val_loss: 1.6954 - val_acc: 0.2684
Epoch 5/30
48/48 [==============================] - 13s 272ms/step - batch: 23.5000 - size: 15.8333 - loss: 1.6865 - acc: 0.2645 - val_loss: 1.6749 - val_acc: 0.2684
Epoch 6/30
48/48 [==============================] - 13s 273ms/step - batch: 23.5000 - size: 15.8333 - loss: 1.6664 - acc: 0.2697 - val_loss: 1.6558 - val_acc: 0.2684
Epoch 